In [1]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf,col
import random
import numpy as np

In [2]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.cm as cm
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from sklearn import decomposition

In [3]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark Clustering with scikit-learn") \
        .getOrCreate()
    return spark
spark = init_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/26 22:13:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/26 22:13:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/26 22:13:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [17]:
#  Read data
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

data = spark.read.csv("../data/lemma.csv", header=True)
function_array = udf(lambda r: r.split("|"), ArrayType(StringType()))
function_toNumerical = udf(lambda r: int(r), IntegerType())
text_lemmas = data.withColumn('finished_lemmas', function_array('text')).drop('text').withColumn('label', function_toNumerical('label'))
print("Number of rows: ",text_lemmas.count())

Py4JJavaError: An error occurred while calling o318.csv.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [ ]:
# Get the Corpus.
# Removing stop words from the text lemmas. 

remover = StopWordsRemover(inputCol="finished_lemmas", outputCol="text")
filtered_df = remover.transform(text_lemmas)

In [ ]:
# Get the Corpus.
# Removing stop words from the text lemmas. 

remover = StopWordsRemover(inputCol="finished_lemmas", outputCol="text")
filtered_df = remover.transform(text_lemmas)

# Create Document-Term Matrix by vectorizing the filtered text.
# - returns the features column: (total nb of words, indices of each word in total vocab, count of each word)

to_vectorize = filtered_df.select('id', 'label', 'text')
cv = CountVectorizer(inputCol="text", outputCol="features")
model_vec = cv.fit(to_vectorize)
result_vec = model_vec.transform(to_vectorize)
print("Total count of vocabulary:", len(model_vec.vocabulary))
selectedData = result_vec.select('id', 'label','features', 'text')

Total count of vocabulary: 53068


In [ ]:
# train k-means
kmeans = KMeans().setK(2).setSeed(10)
model = kmeans.fit(selectedData)

# make predictions
predictions = model.transform(selectedData)

# evaluate clustering
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()

#probably model.data? [[0,10,20],[30,40,60]......]
#for : for: print vol[0],vol[10]
print("Cluster Centers: ")
for center in centers:
    print(center)

22/08/26 22:16:36 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/08/26 22:16:36 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/08/26 22:16:36 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/08/26 22:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1465.1 KiB


22/08/26 22:16:49 WARN DAGScheduler: Broadcasting large task binary with size 1459.3 KiB


22/08/26 22:16:51 WARN DAGScheduler: Broadcasting large task binary with size 1470.9 KiB


22/08/26 22:17:01 WARN DAGScheduler: Broadcasting large task binary with size 1476.9 KiB


Silhouette with squared euclidean distance = 0.9876238309946687
Cluster Centers: 
[7.50674400e-01 6.69611546e-01 5.15308875e-01 ... 6.74399784e-05
 6.74399784e-05 6.74399784e-05]
[9. 3. 8. ... 0. 0. 0.]


In [ ]:
data_pd = selectedData.select('features').rdd
# data_list = data_pd.map(lambda row: list(row['features'])).collect()

Py4JJavaError: An error occurred while calling o315.javaToPython.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.com

In [ ]:
# K-means with Scikit-learn

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
# Instantiate the KMeans models
km = KMeans(n_clusters=2, random_state=10)
# Fit the KMeans model
cluster_labels = km.fit_predict(data_list)
# Calculate Silhoutte Score
score = silhouette_score(data_list, km.labels_, metric='euclidean')
# Compute the silhouette scores for each sample
sample_silhouette_values = silhouette_samples(data_list, cluster_labels)
print('Silhouetter Score: %.3f' % score)

NameError: name 'data_list' is not defined

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


X = data_list
np_X = np.array(X)
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

"""
parameters:
data_list: The features data (vectors) in numpy.array
random_state: The random_state of kmeans
reduce_method: 0 = PCA, 1= SVD+TSNE
"""
def Visualize(np_X,random_state,reduce_method=0):
  range_n_clusters = [2, 3, 4, 5, 6]
  if reduce_method:
    svd = TruncatedSVD(n_components=100, n_iter=10, random_state=random_state)
    svd.fit(np_X)
    svd_concatenated = svd.transform(np_X)
    model = TSNE(n_components=2,learning_rate=250,n_iter=5000).fit(svd_concatenated)
    reduced_=model.fit_transform(svd_concatenated)
  else:
    pca = decomposition.PCA(n_components=2)
    model=pca.fit(np_X)
    reduced_ = pca.transform(np_X)

  for n_clusters in range_n_clusters:
      # Create a subplot with 1 row and 2 columns
      fig, (ax1, ax2) = plt.subplots(1, 2)
      fig.set_size_inches(18, 7)

      # The 1st subplot is the silhouette plot
      # The silhouette coefficient can range from -1, 1 but in this example all
      # lie within [-0.1, 1]
      ax1.set_xlim([-0.1, 1])
      # The (n_clusters+1)*10 is for inserting blank space between silhouette
      # plots of individual clusters, to demarcate them clearly.
      ax1.set_ylim([0, len(reduced_) + (n_clusters + 1) * 10])

      # Initialize the clusterer with n_clusters value and a random generator
      # seed of 10 for reproducibility.
      # Instantiate the KMeans models
      km = KMeans(n_clusters=n_clusters, random_state=random_state)
      # Fit the KMeans model
      km.fit(reduced_)
      cluster_labels=km.predict(reduced_)
      # Calculate Silhoutte Score
      silhouette_avg = silhouette_score(reduced_, km.labels_, metric='euclidean')
      print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)
      # Compute the silhouette scores for each sample
      sample_silhouette_values = silhouette_samples(reduced_, cluster_labels)
      print('Silhouetter Score: %.3f' % silhouette_avg)

      # The silhouette_score gives the average value for all the samples.
      # This gives a perspective into the density and separation of the formed
      # clusters

      y_lower = 10
      for i in range(n_clusters):
          # Aggregate the silhouette scores for samples belonging to
          # cluster i, and sort them
          ith_cluster_silhouette_values = \
              sample_silhouette_values[cluster_labels == i]

          ith_cluster_silhouette_values.sort()

          size_cluster_i = ith_cluster_silhouette_values.shape[0]
          y_upper = y_lower + size_cluster_i

          color = cm.nipy_spectral(float(i) / n_clusters)
          ax1.fill_betweenx(np.arange(y_lower, y_upper),
                            0, ith_cluster_silhouette_values,
                            facecolor=color, edgecolor=color, alpha=0.7)

          # Label the silhouette plots with their cluster numbers at the middle
          ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
          

          # Compute the new y_lower for next plot
          y_lower = y_upper + 10  # 10 for the 0 samples
          

      """
      Plotting
      """
      ax1.set_title("The silhouette plot for the various clusters.")
      ax1.set_xlabel("The silhouette coefficient values")
      ax1.set_ylabel("Cluster label")

      # The vertical line for average silhouette score of all the values
      ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

      ax1.set_yticks([])  # Clear the yaxis labels / ticks
      ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

      # 2nd Plot showing the actual clusters formed
      colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
      ax2.scatter(reduced_[:, 0], reduced_[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                  c=colors, edgecolor='k')

      # Labeling the clusters
      centers = km.cluster_centers_
      # Draw white circles at cluster centers
      
      for i, c in enumerate(centers):
          ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                      s=50, edgecolor='k')
      
      ax2.set_title("The visualization of the clustered data.")
      ax2.set_xlabel("Feature space for the 1st feature")
      ax2.set_ylabel("Feature space for the 2nd feature")

      plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                    "with n_clusters = %d" % n_clusters),
                  fontsize=14, fontweight='bold')

  plt.show()

In [ ]:
Visualize(np_X,10,1)

In [ ]:
Visualize(np_X,10,0)